# 0. Provision our SCNs

In [1]:
from platform_api import platform_api
from smart_broker_api import session

import time
import warnings

# We filter out because our certs are self-signed for now
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

PLATFORM_IP = "172.20.100.8"
PLATFORM_USER = "admin@secureailabs.com"
FEDERATION_NAME = "KCA Consortium"
# PASSWORD = getpass.getpass(prompt="Researcher password: ")
PASSWORD = "SailPassword@123"
platform_session = platform_api.login(PLATFORM_USER, PASSWORD, PLATFORM_IP)

platform_api.provision_federation_by_name(platform_session, FEDERATION_NAME)

federation_session = None
while federation_session is None:
    time.sleep(30)
    federation_session = platform_api.connect_to_federation(platform_session, FEDERATION_NAME)

session = session.Session(federation_session["ip"], federation_session["port"], True)

print(f"Federation is up and running at {session.ip}:{session.port}")

Id for federation is 4af40c28-d689-4ce4-9b30-165ae3999a63
{'data_federation_id': '4af40c28-d689-4ce4-9b30-165ae3999a63', 'secure_computation_nodes_size': 'Standard_D4s_v4'}
Provision id 723284d0-9cdb-4042-a753-9eebc4eaed2d
Smart broker id: fbd299b9-c983-4fbd-ad73-29ffcc195b56
Federation is being provisioned...
Federation is being provisioned...
Federation is being provisioned...
Federation is being provisioned...
Federation is being provisioned...
Federation is being provisioned...
Federation is being provisioned...
Federation is being provisioned...
Federation is up and running at 172.20.200.10:8000


In [ ]:
from platform_api import platform_api

import time
import warnings

# We filter out because our certs are self-signed for now
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

PLATFORM_IP = "172.20.100.12"
PLATFORM_USER = "admin@secureailabs.com"
FEDERATION_NAME = "KCA Consortium"
# PASSWORD = getpass.getpass(prompt="Researcher password: ")
PASSWORD = "SailPassword@123"
platform_session = platform_api.login(PLATFORM_USER, PASSWORD, PLATFORM_IP)

ds_list = platform_api.get_federation_datasets_by_name(platform_session, FEDERATION_NAME)

print(ds_list)

# 1. Aggregation Model Creation

In [2]:
from smart_broker_api import data_model_api

DATA_FRAME_NAME = "data_frame_demo"

data_model_id = data_model_api.create_date_frame(session, DATA_FRAME_NAME)
data_model_api.data_frame_add_series(session, data_model_id, "bmi_mean", "Observation:Body Mass Index", "AgregatorIntervalMean")
data_model_api.data_frame_add_series(session, data_model_id, "bmi_first", "Observation:Body Mass Index", "AgregatorIntervalFirstOccurance")
data_model_api.data_frame_add_series(session, data_model_id, "bmi_last", "Observation:Body Mass Index", "AgregatorIntervalLastOccurance")
data_model_tabular_id = data_model_api.create_tabular_data(session)
data_model_tabular_id = data_model_api.tabular_add_dataframe(session, data_model_id, data_model_tabular_id)

AttributeError: 'Session' object has no attribute 'get_url'

# 2. Flatten / Aggregate data from Aggregation model

In [ ]:
from smart_broker_api import data_api

dataset_federation_id = "a892f738-4f6f-11ed-bdc3-0242ac120002"
dataset_federation_name = "r4sep2019_csvv1_20_1"

longitudinal_id = data_api.read_longitudinal_fhirv1(session)
tabular_dataset_id = data_api.parse_dataset_tabular_from_longitudinal(session, longitudinal_id, dataset_federation_id, dataset_federation_name, data_model_tabular_id)
data_frame_id = data_api.data_frame_tabular_select_data_frame(session, tabular_dataset_id, DATA_FRAME_NAME)

# 3. Remove NaNs from data frame

In [ ]:
from smart_broker_api import preprocessing_api

no_nan_data_frame_id = preprocessing_api.drop_na_data_frame(session, data_frame_id)

# 4. Grab series to work with in Statistics Lib

In [ ]:
series_1_id = data_api.data_frame_select_series(session, no_nan_data_frame_id, "bmi_mean")
series_2_id = data_api.data_frame_select_series(session, no_nan_data_frame_id, "bmi_last")

print(series_1_id+" , "+series_2_id)

# 5. Cohort selection

In [ ]:
from smart_broker_api import preprocessing_api
from smart_broker_api import statistics_api

query = "bmi_first > bmi_last"
query_dataframe = preprocessing_api.query_data_frame(session, no_nan_data_frame_id, query)
query_series = data_api.data_frame_select_series(session, query_dataframe, "bmi_last")

print(f"Patients whose BMI decreased during study: {statistics_api.count(session,  query_series)}")

query = "bmi_first < bmi_last"
query_dataframe = preprocessing_api.query_data_frame(session, no_nan_data_frame_id, query)
query_series = data_api.data_frame_select_series(session, query_dataframe, "bmi_last")

print(f"Patients whose BMI increased during study: {statistics_api.count(session,  query_series)}")

query = "bmi_first - bmi_last > 0.5"
query_dataframe = preprocessing_api.query_data_frame(session, no_nan_data_frame_id, query)
query_series = data_api.data_frame_select_series(session, query_dataframe, "bmi_last")

print(f"Patients whose BMI significantly increased during study: {statistics_api.count(session,  query_series)}")

print(f"Mean BMI of patients whose BMI increased: {statistics_api.mean(session, query_series)} ")

# 6. Run Statistics Library

In [ ]:
from smart_broker_api import statistics_api
from smart_broker_api import visualization_api
import plotly.graph_objects as go

type_distribution="normalunit"
type_ranking="cdf"
alternative = "two-sided"
print(statistics_api.count(session,  series_1_id))
print(statistics_api.mean(session,  series_1_id))
print(statistics_api.chisquare(session,  series_1_id, series_2_id))
print(statistics_api.kolmogorovSmirnovTest(session,  series_1_id, type_distribution, type_ranking))
print(statistics_api.kurtosis(session,  series_1_id))
print(statistics_api.levene_test(session,  series_1_id, series_2_id))
print(statistics_api.mann_whitney_u_test(session,  series_1_id, series_2_id, alternative, type_ranking))
print(statistics_api.min_max(session,  series_1_id))
print(statistics_api.paired_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_api.pearson(session,  series_1_id, series_2_id, alternative))
print(statistics_api.skewness(session,  series_1_id))
print(statistics_api.spearman(session,  series_1_id, series_2_id, alternative, type_ranking))
print(statistics_api.student_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_api.variance(session,  series_1_id))
print(statistics_api.welch_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_api.wilcoxon_signed_rank_test(session,  series_1_id, series_2_id, alternative, type_ranking))
print(statistics_api.wilcoxon_signed_rank_test(session,  series_1_id, series_2_id, alternative, type_ranking))



dict_of_fig = visualization_api.histogram(session, series_1_id, 20)
fig = go.Figure(dict_of_fig["figure"])
fig.show()


dict_of_fig = visualization_api.kernel_density_estimation(session, series_1_id, 2)
fig = go.Figure(dict_of_fig["figure"])
fig.show()




# 7. Deprovision our Federation

In [3]:

print(platform_session.provision_clusters)
platform_api.deprovision_federation_by_name(platform_session, FEDERATION_NAME)


{'KCA Consortium': {'provision_id': '723284d0-9cdb-4042-a753-9eebc4eaed2d', 'broker_id': 'fbd299b9-c983-4fbd-ad73-29ffcc195b56'}}
Deprovision ID is 723284d0-9cdb-4042-a753-9eebc4eaed2d


In [ ]:
from platform_api import platform_api

PROVISION_ID = "0de43d62-fb01-4c34-aaae-3f77a73f71e7"
PASSWORD="SailPassword@123"

platform_session = platform_api.login("admin@secureailabs.com", PASSWORD, "172.20.100.12")

platform_api.deprovision_federation_by_id(platform_session, PROVISION_ID)